In [ ]:
import numpy as np
import tensorflow as tf
from nltk.corpus import gutenberg
import nltk

# Load the trained model
model = tf.keras.models.load_model('text_generation_lstm.h5')

# Load the same dataset used for training
nltk.download('gutenberg')
text = gutenberg.raw('shakespeare-hamlet.txt').lower()

# Create character mappings
chars = sorted(list(set(text)))
char_to_int = {char: number for number, char in enumerate(chars)}
int_to_char = {number: char for number, char in enumerate(chars)}

In [ ]:
def generate_text(seed_text, length=500, temperature=1.0):
    generated_text = seed_text
    for _ in range(length):
        # Convert seed text to one-hot encoding
        input_seq = np.zeros((1, len(seed_text), len(chars)))
        for t, char in enumerate(seed_text):
            if char in char_to_int:
                input_seq[0, t, char_to_int[char]] = 1

        # Predict next character
        predictions = model.predict(input_seq, verbose=0)[0]

        # Apply temperature to predictions
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        probabilities = exp_preds / np.sum(exp_preds)

        # Sample a character based on probabilities
        next_index = np.random.choice(len(chars), p=probabilities)
        next_char = int_to_char[next_index]

        # Append next char and update seed text
        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text

In [ ]:
# Example seed text from Hamlet
seed = "to be or not to be that is the question"

# Generate text with different temperature settings
print("\nLow Temperature (Predictable Output):")
print(generate_text(seed, length=300, temperature=0.2))

print("\nHigh Temperature (More Creative Output):")
print(generate_text(seed, length=300, temperature=1.0))